In [ ]:
import pandas as pd
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import keras
import datetime
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import pickle
import re
import tensorflow as tf
from Modelling_Module.model_evaluation_lstm import most_common, reveal_bias, add_bias_labels, create_reqset_dict, get_reqset_bias, get_app_bias_error

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

tf.keras.backend.set_session(session)

# LSTM model stability test

In [ ]:
path = './Datasets/'
model_path = './Models/'

# Read in config file - column list

cumsum_cols_p = open(path + 'cumsum_cols.pkl', 'rb')
cumsum_cols = pickle.load(cumsum_cols_p)

In [ ]:
# Read in prepared datasets

X_val_p = open(path + 'X_val.pkl', 'rb')
X_val = pickle.load(X_val_p)

sub_y_val_p = open(path + 'y_val.pkl', 'rb')
sub_y_val = pickle.load(sub_y_val_p)

X_train_p = open(path + 'X_train.pkl', 'rb')
X = pickle.load(X_train_p)

y_train_p = open(path + 'y_train.pkl', 'rb')
sub_y = pickle.load(y_train_p)

X_test_p = open(path + 'X_test.pkl', 'rb')
X_test = pickle.load(X_test_p)

y_test_p = open(path + 'y_test.pkl', 'rb')
sub_y_test = pickle.load(y_test_p)

## Run model five times to check stability

In [ ]:
'''
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
'''

# stability search

# collect data across multiple repeats
train = pd.DataFrame()
val = pd.DataFrame()
holdout = pd.DataFrame()
app_bias_error = pd.DataFrame()

hold_lst = []
app_lst = []
for i in range(5):
    # define model
    
    model = Sequential()
    model.add(LSTM(125, return_sequences=True, input_shape=(1000, 129)))
    model.add(LSTM(125, return_sequences=True))
    model.add(LSTM(125))
    model.add(Dense(129))
    
    # compile model
    model.compile(loss='mae', optimizer=keras.optimizers.Adam(lr=0.01))
    # fit model
    history = model.fit(X, sub_y, batch_size=128, epochs=1, validation_data=(X_val, sub_y_val), shuffle=False)
    # story history
    train[str(i)] = history.history['loss']
    val[str(i)] = history.history['val_loss']
    
    # evaluate model by MAE
    
    test_MAE = model.evaluate(X_test, sub_y_test, verbose=0)

    holdout[str(i)] = [test_MAE]

    # compared predicted request set "app bias" with actual "app bias"

    app_bias_list = get_app_bias_error(X_test, sub_y_test, model, cumsum_cols)

    app_bias_df = pd.DataFrame(app_bias_list)

    app_bias_e = app_bias_df.mean().mean()
    
    app_bias_error[str(i)] = [app_bias_e]

In [ ]:
# plot train and validation loss across multiple runs

plt.plot(train.T, color='blue', label='train')
plt.plot(val.T, color='orange', label='validation')
plt.plot(holdout.T, color='green', label='test')
plt.plot(app_bias_error.T, color='red', label='app_bias_error')
plt.legend()

plt.title('Final LSTM model stability run')
plt.ylabel('loss')
plt.xlabel('run')
plt.show()

In [ ]:
final_stable_model_list = [train, val, holdout, app_bias_error]

with open('final_LSTM_model_stability', 'wb') as f:
                        pickle.dump(final_stable_model_list, f)